# **QUESTION01**

In [13]:
def diagnose_disease(user_symptoms, user_test_results):
    disease_data = {
        "Acute Appendicitis": {
            "symptoms": {"fever", "pain in abdomen", "vomiting"},
            "tests": {"tlc high", "dlc neutrophils high", "esr high"},
            "treatment": "Surgery"
        },
        "Pneumonia": {
            "symptoms": {"fever", "cough", "chest pain"},
            "tests": {"tlc high", "dlc neutrophils high", "esr high", "x-ray shows pneumonic patch"},
            "treatment": "Antibiotics"
        },
        "Acute Tonsillitis": {
            "symptoms": {"fever", "cough"},
            "tests": {"red enlarged tonsils", "pus in tonsils"},
            "treatment": "Anti-allergic + Paracetamol. If not cured: Add antibiotics orally. If still not cured: Add IV antibiotics."
        },
        "Migraine": {
            "symptoms": {"headache", "nausea", "sensitivity to light"},
            "tests": set(),
            "treatment": "Pain relievers, Rest in dark and quiet room."
        },
        "Diabetes": {
            "symptoms": {"increased thirst", "frequent urination", "fatigue"},
            "tests": {"blood sugar high"},
            "treatment": "Insulin therapy, Lifestyle changes."
        }
    }

    user_symptoms = {s.lower().strip() for s in user_symptoms}
    user_test_results = {t.lower().strip() for t in user_test_results}

    for disease, details in disease_data.items():
        symptom_match = all(symptom in user_symptoms for symptom in details["symptoms"])
        test_match = all(test in user_test_results for test in details["tests"])
        if symptom_match and test_match:
            return f"Diagnosed Disease: {disease}\nRecommended Treatment: {details['treatment']}"

    potential_diseases = []
    for disease, details in disease_data.items():
        matched_symptoms = len(details["symptoms"].intersection(user_symptoms))
        matched_tests = len(details["tests"].intersection(user_test_results))
        total_criteria = len(details["symptoms"]) + len(details["tests"])
        if total_criteria > 0:
            match_percentage = (matched_symptoms + matched_tests) / total_criteria * 100
            if match_percentage > 0:
                potential_diseases.append((disease, match_percentage))

    potential_diseases.sort(key=lambda x: x[1], reverse=True)

    if potential_diseases:
        result = "No exact match found. Potential matches:\n"
        for disease, percentage in potential_diseases:
            result += f"- {disease} ({percentage:.1f}% match)\n"
        return result
    else:
        return "No matching disease found. Please consult a doctor."

symptoms_input = input("Enter symptoms (comma-separated): ").split(",")
test_results_input = input("Enter test results (comma-separated): ").split(",")

symptoms = {s.strip() for s in symptoms_input}
test_results = {t.strip() for t in test_results_input}

result = diagnose_disease(symptoms, test_results)
print(result)


Enter symptoms (comma-separated): fever,cough
Enter test results (comma-separated): tlc high
No exact match found. Potential matches:
- Acute Tonsillitis (50.0% match)
- Pneumonia (42.9% match)
- Acute Appendicitis (33.3% match)



# **QUESTION02**

In [5]:
import random

class SimpleAgent:
    def __init__(self):
        self.world = [
            ["Start", "Empty", "Pit", "Empty"],
            ["Empty", "Pit", "Wumpus", "Empty"],
            ["Empty", "Gold", "Empty", "Pit"],
            ["Empty", "Pit", "Empty", "Empty"]
        ]
        self.pos = (0, 0)
        self.gold = False
        self.steps = 0
        self.state = [row[:] for row in self.world]
        self.path = []
        self.seen = set()
        self.seen.add(self.pos)

    def check_surroundings(self, where):
        x, y = where
        places = []
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < 4 and 0 <= ny < 4:
                places.append((nx, ny, self.state[nx][ny]))
        random.shuffle(places)
        return places

    def take_gold(self):
        x, y = self.pos
        if self.state[x][y] == "Gold":
            self.gold = True
            self.state[x][y] = "Empty"
            print("Gold taken")

    def go(self, way):
        x, y = self.pos
        if way == "up" and x > 0:
            self.pos = (x - 1, y)
        elif way == "down" and x < 3:
            self.pos = (x + 1, y)
        elif way == "left" and y > 0:
            self.pos = (x, y - 1)
        elif way == "right" and y < 3:
            self.pos = (x, y + 1)
        self.path.append((x, y))
        self.steps += 1
        self.seen.add(self.pos)

    def go_home(self):
        print("Going home")
        path_copy = self.path.copy()
        while self.pos != (0, 0):
            if not path_copy:
                print("No way back")
                break
            self.pos = path_copy.pop()
            print("Home path", self.pos)
        print("Home path full", self.path)

    def choose_move(self):
        x, y = self.pos
        if self.state[x][y] == "Gold":
            return "take_gold"

        places = self.check_surroundings(self.pos)
        for nx, ny, thing in places:
            if (thing == "Empty" or thing == "Gold") and (nx, ny) not in self.seen:
                if nx < x:
                    return "go_up"
                elif nx > x:
                    return "go_down"
                elif ny < y:
                    return "go_left"
                elif ny > y:
                    return "go_right"

        if self.path:
            return "back"

        return "stay"

    def back(self):
        if self.path:
            self.pos = self.path.pop()
            print("Going back", self.pos)
        else:
            print("Stuck, nowhere to go back")

    def show_world(self):
        for i in range(4):
            row = []
            for j in range(4):
                if (i, j) == self.pos:
                    row.append("A")
                else:
                    row.append(self.state[i][j])
            print(row)

    def play(self):
        while self.steps < 50:
            print("Now at", self.pos)
            print("World is like this")
            self.show_world()
            move = self.choose_move()
            print("Doing", move)
            if move == "take_gold":
                self.take_gold()
                self.go_home()
                break
            elif move.startswith("go"):
                self.go(move.split("_")[1])
            elif move == "back":
                self.back()
            elif move == "stay":
                print("No more moves, stopping")
                break
            input("Press Enter...")
        if self.gold and self.pos == (0, 0):
            print("Win! Got gold and reached start")
        else:
            print("Lost, no gold in 50 steps")

if __name__ == "__main__":
    game = SimpleAgent()
    game.play()


Now at (0, 0)
World is like this
['A', 'Empty', 'Pit', 'Empty']
['Empty', 'Pit', 'Wumpus', 'Empty']
['Empty', 'Gold', 'Empty', 'Pit']
['Empty', 'Pit', 'Empty', 'Empty']
Doing go_down
Press Enter...
Now at (1, 0)
World is like this
['Start', 'Empty', 'Pit', 'Empty']
['A', 'Pit', 'Wumpus', 'Empty']
['Empty', 'Gold', 'Empty', 'Pit']
['Empty', 'Pit', 'Empty', 'Empty']
Doing go_down
Press Enter...
Now at (2, 0)
World is like this
['Start', 'Empty', 'Pit', 'Empty']
['Empty', 'Pit', 'Wumpus', 'Empty']
['A', 'Gold', 'Empty', 'Pit']
['Empty', 'Pit', 'Empty', 'Empty']
Doing go_down
Press Enter...
Now at (3, 0)
World is like this
['Start', 'Empty', 'Pit', 'Empty']
['Empty', 'Pit', 'Wumpus', 'Empty']
['Empty', 'Gold', 'Empty', 'Pit']
['A', 'Pit', 'Empty', 'Empty']
Doing back
Going back (2, 0)
Press Enter...
Now at (2, 0)
World is like this
['Start', 'Empty', 'Pit', 'Empty']
['Empty', 'Pit', 'Wumpus', 'Empty']
['A', 'Gold', 'Empty', 'Pit']
['Empty', 'Pit', 'Empty', 'Empty']
Doing go_right
Press Ent

# **QUESTION03**

In [9]:
import math

grid = [
    ['P', 'C', 'F', 'F'],
    ['F', 'G', 'F', 'F'],
    ['C', 'F', 'C', 'G'],
    ['F', 'F', 'G', 'F']
]

pacman_pos = (0, 0)
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def print_grid():
    for row in grid:
        print(row)
    print()

def is_valid_move(x, y):
    return 0 <= x < 4 and 0 <= y < 4 and grid[x][y] != 'W'

def find_nearest_pellet(pacman_pos):
    min_distance = math.inf
    target = None
    for i in range(4):
        for j in range(4):
            if grid[i][j] in ['F', 'C']:
                distance = abs(pacman_pos[0] - i) + abs(pacman_pos[1] - j)
                if distance < min_distance:
                    min_distance = distance
                    target = (i, j)
    return target

def goal_based_agent():
    global pacman_pos
    power_pellet_active = False
    food_pellets_remaining = sum(row.count('F') for row in grid)
    power_pellets_remaining = sum(row.count('C') for row in grid)

    while food_pellets_remaining > 0 or power_pellets_remaining > 0:
        print("Current Grid:")
        print_grid()
        print(f"Pacman Position: {pacman_pos}")
        print(f"Food Pellets Remaining: {food_pellets_remaining}")
        print(f"Power Pellets Remaining: {power_pellets_remaining}\n")

        target = find_nearest_pellet(pacman_pos)
        if not target:
            print("No more pellets to consume!")
            break

        dx, dy = target[0] - pacman_pos[0], target[1] - pacman_pos[1]
        next_move = (pacman_pos[0] + (1 if dx > 0 else -1), pacman_pos[1]) if dx != 0 else (pacman_pos[0], pacman_pos[1] + (1 if dy > 0 else -1))

        if not is_valid_move(*next_move):
            print("Invalid move! Pacman is stuck.")
            break

        grid[pacman_pos[0]][pacman_pos[1]] = 'E'
        pacman_pos = next_move

        if grid[pacman_pos[0]][pacman_pos[1]] == 'G' and not power_pellet_active:
            print("Pacman was destroyed by a ghost! Game Over.")
            break

        if grid[pacman_pos[0]][pacman_pos[1]] == 'F':
            food_pellets_remaining -= 1
        elif grid[pacman_pos[0]][pacman_pos[1]] == 'C':
            power_pellets_remaining -= 1
            power_pellet_active = True

        grid[pacman_pos[0]][pacman_pos[1]] = 'P'

    if food_pellets_remaining == 0 and power_pellets_remaining == 0:
        print("Congratulations! Pacman consumed all pellets.")

goal_based_agent()


Current Grid:
['P', 'C', 'F', 'F']
['F', 'G', 'F', 'F']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position: (0, 0)
Food Pellets Remaining: 9
Power Pellets Remaining: 3

Current Grid:
['E', 'P', 'F', 'F']
['F', 'G', 'F', 'F']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position: (0, 1)
Food Pellets Remaining: 9
Power Pellets Remaining: 2

Current Grid:
['E', 'E', 'P', 'F']
['F', 'G', 'F', 'F']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position: (0, 2)
Food Pellets Remaining: 8
Power Pellets Remaining: 2

Current Grid:
['E', 'E', 'E', 'P']
['F', 'G', 'F', 'F']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position: (0, 3)
Food Pellets Remaining: 7
Power Pellets Remaining: 2

Current Grid:
['E', 'E', 'E', 'E']
['F', 'G', 'F', 'P']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position: (1, 3)
Food Pellets Remaining: 6
Power Pellets Remaining: 2

Current Grid:
['E', 'E', 'E', 'E']
['F', 'G', 'P', 'E']
['C', 'F', 'C', 'G']
['F', 'F', 'G', 'F']

Pacman Position:

# **QUESTION04**

In [5]:
def dfs(start, get_moves, is_complete):
    stack = [(start, [], 0)]
    visited = set()

    while stack:
        state, path, cost = stack.pop()
        location, rooms_served = state
        state_key = (location, tuple(sorted(rooms_served.items())))

        if is_complete(state):
            return path, cost

        if state_key in visited:
            continue
        visited.add(state_key)

        for move, next_state, move_cost in get_moves(state):
            stack.append((next_state, path + [move], cost + move_cost))

    return None, float('inf')

def iddfs(start, get_moves, is_complete):
    depth = 0
    while True:
        result = dls(start, get_moves, is_complete, depth)
        if result is not None:
            return result
        depth += 1

def dls(state, get_moves, is_complete, depth, path=[], cost=0):
    if depth == 0:
        return (path, cost) if is_complete(state) else None

    location, rooms_served = state
    for move, next_state, move_cost in get_moves(state):
        result = dls(next_state, get_moves, is_complete, depth - 1, path + [move], cost + move_cost)
        if result is not None:
            return result

    return None

def is_complete(state):
    _, rooms_served = state
    return all(rooms_served.values())

def get_moves(state):
    location, rooms_served = state
    moves = []

    if location == 'Service':
        for room in ['A', 'B', 'C']:
            if not rooms_served[room]:
                updated_rooms = rooms_served.copy()
                updated_rooms[room] = True
                cost = 5 if room in ['A', 'B'] else 1
                moves.append((f"Move to {room} and Serve", (room, updated_rooms), cost))
    else:
        moves.append(("Return to Service Room", ('Service', rooms_served), 0))

    return moves

initial_state = ('Service', {'A': False, 'B': False, 'C': False})

dfs_path, dfs_cost = dfs(initial_state, get_moves, is_complete)
print("DFS Path:", dfs_path if dfs_path else "No solution found")
print("DFS Total Cost:", dfs_cost)

iddfs_path, iddfs_cost = iddfs(initial_state, get_moves, is_complete)
print("\nIDDFS Path:", iddfs_path if iddfs_path else "No solution found")
print("IDDFS Total Cost:", iddfs_cost)

DFS Path: ['Move to C and Serve', 'Return to Service Room', 'Move to B and Serve', 'Return to Service Room', 'Move to A and Serve']
DFS Total Cost: 11

IDDFS Path: ['Move to A and Serve', 'Return to Service Room', 'Move to B and Serve', 'Return to Service Room', 'Move to C and Serve']
IDDFS Total Cost: 11


# **QUESTION05**

In [7]:
from heapq import heappop, heappush
from itertools import combinations

def is_goal_state(state):
    return len(state[0]) == 0

def get_successors(state):
    successors_list = []
    left_side, right_side, is_flashlight_left, current_cost, move_path = state

    if is_flashlight_left:
        for pair in combinations(left_side, 2):
            new_left = [person for person in left_side if person not in pair]
            new_right = right_side + list(pair)
            time_taken = max(pair)
            new_path = move_path + [(pair, "cross", time_taken)]
            successors_list.append((new_left, new_right, False, current_cost + time_taken, new_path))
    else:
        for person in right_side:
            new_left = left_side + [person]
            new_right = [p for p in right_side if p != person]
            new_path = move_path + [((person,), "return", person)]
            successors_list.append((new_left, new_right, True, current_cost + person, new_path))

    return successors_list

def uniform_cost_search(crossing_times):
    initial_state = (crossing_times, [], True, 0, [])
    priority_queue, visited_states, total_states = [(0, initial_state)], set(), 0

    while priority_queue:
        _, current_state = heappop(priority_queue)
        total_states += 1
        left_side, right_side, is_flashlight_left, current_cost, move_path = current_state
        print("State", total_states, "Left:", left_side, "Right:", right_side, "Flashlight:", "Left" if is_flashlight_left else "Right", "Cost:", current_cost)

        if is_goal_state(current_state):
            return move_path, current_cost, total_states

        state_key = (tuple(sorted(left_side)), tuple(sorted(right_side)), is_flashlight_left)
        if state_key in visited_states:
            continue
        visited_states.add(state_key)

        for successor in get_successors(current_state):
            heappush(priority_queue, (successor[3], successor))

    return None, float('inf'), total_states

crossing_times = list(map(int, input("Enter crossing times: ").split()))
path, total_time, states_visited = uniform_cost_search(crossing_times)

if path:
    print("Optimal moves:")
    for step, (people, action, time_taken) in enumerate(path, 1):
        print(step, people, action, "Time:", time_taken)
    print("Total time:", total_time, "minutes")
    print("States visited:", states_visited)
else:
    print("No solution found.")


Enter crossing times: 1 2 5 10
State 1 Left: [1, 2, 5, 10] Right: [] Flashlight: Left Cost: 0
State 2 Left: [5, 10] Right: [1, 2] Flashlight: Right Cost: 2
State 3 Left: [5, 10, 1] Right: [2] Flashlight: Left Cost: 3
State 4 Left: [5, 10, 2] Right: [1] Flashlight: Left Cost: 4
State 5 Left: [1, 10] Right: [2, 5] Flashlight: Right Cost: 5
State 6 Left: [2, 10] Right: [1, 5] Flashlight: Right Cost: 5
State 7 Left: [2, 10, 1] Right: [5] Flashlight: Left Cost: 6
State 8 Left: [1, 10, 2] Right: [5] Flashlight: Left Cost: 7
State 9 Left: [10] Right: [2, 5, 1] Flashlight: Right Cost: 8
State 10 Left: [10] Right: [5, 2, 1] Flashlight: Right Cost: 8
State 11 Left: [10] Right: [1, 5, 2] Flashlight: Right Cost: 9
State 12 Left: [10, 1] Right: [2, 5] Flashlight: Left Cost: 9
State 13 Left: [1, 2] Right: [5, 10] Flashlight: Right Cost: 10
State 14 Left: [1, 5] Right: [2, 10] Flashlight: Right Cost: 10
State 15 Left: [1, 10, 5] Right: [2] Flashlight: Left Cost: 10
State 16 Left: [2, 5] Right: [1, 10